In [1]:
import pandas as pd
import numpy as np
import os
import json
from rq import Queue
from redis import Redis

import time
import parse_utils
from extract_info_from_input_8 import extract_info_from_df
import copy
import db_utils
import search_utils
from search_theme_color import theme_compare
import match_utils



In [2]:
Queue('ppt_server', connection=Redis())

Queue('ppt_server')

In [3]:
help(Queue.enqueue)

Help on function enqueue in module rq.queue:

enqueue(self, f, *args, **kwargs)
    Creates a job to represent the delayed function call and enqueues it.



In [4]:
text_info_columns = ['file_name','name','info_key',
                         "font_size","color","margin_right","letter_spacing",
                         "text_position","punctuation_wrap","margin_left","font_name",
                         "font_style","margin_bottom","content","font_variant",
                         "font_weight","font_size_asian","text_underline_style","text_indent",
                         "text_shadow","text_line_through_style","text_align","writing_mode",
                         "font_name_asian","language","margin_top","text_transform",
                         "line_height"]

common_columns = ['subject', 'class', 'type', 'name',
                  'width', 'height','file_name', 'type_unique',
                  'key','estimate_title','font_size_estimate_title']
# 这里的shape指的元素
shape_columns = ['shape_height', 'shape_width', 'shape_x', 'shape_y',
                 'shape_area','shape_num','text_num', 'over_shape_flag',
                 'over_shape_num', 'over_shape_area','over_shape_area_rate', 'over_text_flag',
                 'over_text_num', 'over_text_area','over_text_area_rate']
# 这里的text指的是文本框
text_columns = [ 'content_size','content','line', 'slope',
                 'language','font-size-asian', 'font-name-asian', 'font-weight', 'font-name',
                 'font-style', 'font-size'] #这里的language到font-size指的是文本框中的第一个

all_columns = common_columns + shape_columns + text_columns


##################################################
#                  加载数据库
##################################################
version = 'v4'
# db_path = '/home/nd/ppt_server/ppt_auto_layout/datasets/data_ppt'
db_path =  r'E:\Dataset\datasets\datasets\data_ppt'
file1_db = '29_work_template'
image_db = pd.read_csv(os.path.join(db_path,version,'image',file1_db,'ppt_color.csv'))
layout_db_list = []
for i in range(6):
    layout_db_list.append(pd.read_csv(os.path.join(db_path,version,'db',file1_db,'ppt_scd_' + str(i) + '.csv')))
all_db = pd.read_csv(os.path.join(db_path,version,'all',file1_db,'all.csv'))
text_db = pd.read_csv(os.path.join(db_path,version,'all',file1_db,'text.csv'))


In [5]:
for df in layout_db_list:
    print(df.shape)

(2435, 39)
(163, 39)
(159, 39)
(123, 39)
(36, 39)
(10, 39)


In [6]:

with open(r'E:\ND_work\ppt_server-master\ppt_server-master\datasets\test\test1\lo0.xml.json',encoding='utf-8') as f:
    file_json = json.loads(f.read())
# a

In [7]:
# data_path = 
# image_file_path = r'E:\ND_work\ppt_server-master\ppt_server-master\datasets\test\test1\lobin\black3.png'


In [8]:

path= r'E:\ND_work\ppt_server-master\ppt_server-master\datasets\test\test1\lobin.zip'
image_file_path = path.replace('.zip','')
data_path = os.path.split(path)[0]
image_file_path,data_path

('E:\\ND_work\\ppt_server-master\\ppt_server-master\\datasets\\test\\test1\\lobin',
 'E:\\ND_work\\ppt_server-master\\ppt_server-master\\datasets\\test\\test1')

In [9]:
pd.read_csv(os.path.join(data_path, 'info',  'lo0'))

,subject,class,type,name,width,height,file_name,type_unique,key,estimate_title,...,line,slope,language,font-size-asian,font-name-asian,font-weight,font-name,font-style,font-size,pair
0,texts,TextShape,文本框,文本框 7,33.867,19.05,E:\ND_work\ppt_server-master\ppt_server-master...,文本框,0,0,...,1,2.241898,zh,18pt,微软雅黑1,normal,Arial1,normal,18pt,no
1,shape,GraphicObjectShape,图片,图片 1,33.867,19.05,E:\ND_work\ppt_server-master\ppt_server-master...,图片,0,0,...,0,1.730190,none,none,none,none,none,none,none,no


In [11]:

a,b = pd.DataFrame(),pd.DataFrame()
def search_ppt_and_elements_v4_server(file_json,data_path,image_file_path):
    start_time = time.time()
    one_levels = ['data_ppt', 'pic', 'json', 'info']
    pic_levels = ['text', 'index']
    
    ##################################################
    #                  新建文件夹
    ##################################################
    parse_utils.make_parse_filedir_server(data_path,one_levels,pic_levels)
    print('new dir....')
    ##################################################
    #                  解析json文件
    ##################################################
    file4 = 'lo0'
    query_all_info_df, query_text_info_df = parse_utils.parse_json_2_csv_server(data_path,file4,file_json,one_levels,
                                                                         text_info_columns, all_columns,
                                                                         pic_levels)
    global a,b
    a,b = query_all_info_df, query_text_info_df
    ##################################################
    #        判断待匹配PPT类型，同时修正部分解析结果
    ##################################################

    text_type, tuwen_num, tuwen_lyout, rgraph_type, rgpaph_num = extract_info_from_df(query_all_info_df,query_text_info_df)
    
    print('extract_info_from_df',text_type, tuwen_num, tuwen_lyout, rgraph_type, rgpaph_num)
    
    if 1 in text_type and 3 in text_type:
        query_all_info_df = parse_utils.post_deal_with_content_itemize(query_all_info_df, query_text_info_df)
    text_type, tuwen_num, tuwen_lyout, rgraph_type, rgpaph_num = extract_info_from_df(query_all_info_df,
                                                                                      query_text_info_df)
    query_all_info_df_copy = copy.copy(query_all_info_df)  # 测试
    query_text_info_df_copy = copy.copy(query_text_info_df)  # 测试

    query_all_info_df, query_text_info_df = parse_utils.post_deal_with_title_separation(query_all_info_df,
                                                                                        query_text_info_df)
    text_type, tuwen_num, tuwen_lyout, rgraph_type, rgpaph_num = extract_info_from_df(query_all_info_df,
                                                                                      query_text_info_df)
    print('extract_info_from_df',text_type, tuwen_num, tuwen_lyout, rgraph_type, rgpaph_num)
    ##################################################
    #                  存储
    ##################################################
    query_all_info_df.to_csv(os.path.join(data_path, 'data_ppt',  file4), index=0)
    query_all_info_df.to_csv(os.path.join(data_path, 'info',  file4), index=0)
    
    ##################################################
    #                  待匹配PPT统计量
    ##################################################
    query_df = db_utils.get_stat_db_3(query_all_info_df, 0)
    print('query_df: ', query_df.shape)
#     print(query_df)
    if [5] == rgraph_type:
        query_df = db_utils.post_deal_with_total_structure_csv(query_df)

    ##################################################
    #                  获取对应页面数据库
    ##################################################
    page_type = 0  # 主页
    query_labels = {}
    query_labels['tuwen_layout'] = tuwen_lyout
    query_labels['text_type'] = text_type
    query_labels['rgraph_type'] = rgraph_type
    rgraph_flag = True
    db_df = layout_db_list[page_type]

    ##################################################
    #                  推荐PPT
    ##################################################
    print('推荐PPT')
    print(query_df.shape,query_all_info_df.shape,db_df.shape,query_labels)
    recommend_df, second_flag, query_df, query_all_info_df = search_utils.get_best_match_ppt_v4(query_df,
                                                                                                query_all_info_df,
                                                                                                db_df, query_labels,
                                                                                                rgraph_flag)
    print('共匹配PPT数量', len(recommend_df), '张。')

    ##################################################
    #                  颜色匹配
    ##################################################
    if [0] != tuwen_lyout:
        color = True
    else:
        color = False
    if color:
        recommend_df.loc[:, 'rank_orign_ruler'] = range(len(recommend_df))
        result = pd.merge(recommend_df, image_db, how='inner',
                          on=['file_name'])  # ['file_name','image_num','image_color','image_name']
        result = result[['file_name', 'image_num', 'image_color', 'image_name', 'tuwen_color', 'rank_orign_ruler']]


        input_paths = []
        if 'Shapes' in file_json.keys():
            file_json_shape = file_json['Shapes']
            for one in file_json_shape:
                if one['class'] == 'GraphicObjectShape':
                    input_paths.append(os.path.join(image_file_path,
                                                    one['BinName'][0] + '.png'))

        print('带图片共匹配PPT数量', len(result), '张。')
        db_graphics = result["image_color"]
        db_colors = result["tuwen_color"]
        
        print(input_paths, db_graphics.shape) 
        print('++++++++++++++')
        print(db_colors.shape)  # TODO
        search_id, search_loss = theme_compare(input_paths, db_graphics, db_colors)
        recommend_df_copy = recommend_df.copy()
        recommend_df_copy = recommend_df_copy.reset_index(drop=True)  # 测试观察
        color_result = pd.DataFrame(columns=['file_name', 'loss', 'rank_color'])
        for j, i in enumerate(search_id):
            color_dict = {}
            color_dict['file_name'] = result.iloc[i]['file_name']
            color_dict['loss'] = search_loss[j]
            color_dict['rank_color'] = j
            color_result = color_result.append(color_dict, ignore_index=True)

        result_copy = result.copy()
        result_copy = pd.merge(result_copy, color_result, how='inner', on=['file_name'])
        result_copy.loc[:, 'rank_ruler'] = range(len(result_copy))
        result_copy.loc[:, 'rank_all'] = result_copy['rank_ruler'] + result_copy['rank_color']
        result_all = result_copy.sort_values(['rank_all', 'rank_ruler', 'rank_color'], ascending=[True, True, True])
        result_all_index = result_all.index
        for i in range(len(result_all)):
            print(result_all.loc[result_all_index[i]]['file_name'],
                  ' rank_orign_ruler ', result_all.loc[result_all_index[i]]['rank_orign_ruler'],
                  ' rank_all ', result_all.loc[result_all_index[i]]['rank_all'],
                  ' rank_ruler ', result_all.loc[result_all_index[i]]['rank_ruler'],
                  ' rank_color ', result_all.loc[result_all_index[i]]['rank_color'],
                  ' loss ', result_all.loc[result_all_index[i]]['loss'])

        recommend_df = pd.merge(result_all, recommend_df, how='inner', on=['file_name'])
        print('颜色筛选共匹配PPT数量', len(recommend_df), '张。')

    map_json = match_utils.deal_elements_match_v4_server(query_df, query_all_info_df, recommend_df,
                                       second_flag,
                                       data_path, r'E:\Dataset\datasets\datasets\data_ppt',version, file4,
#                                         data_path, r'datasets\data_ppt',version, file4,
                                       file1_db, query_labels, db_df, all_db)
    end_time = time.time()
    print('process time ', str(end_time - start_time))
    print('hello')
    return map_json


search_ppt_and_elements_v4_server(file_json,data_path,image_file_path)

new dir....
extract_info_from_df [0] [1] [3, 1] [0] [0]
extract_info_from_df [0] [1] [3, 1] [0] [0]
query_df:  (1, 31)
推荐PPT
(1, 31) (2, 38) (2435, 39) {'tuwen_layout': [3, 1], 'text_type': [0], 'rgraph_type': [0]}
hello
参数 text_type 0 tuwen_layout 1 rgraph_type 0 匹配 49
参数 text_type 0 tuwen_layout 0 rgraph_type 0 匹配 1
第一次原始匹配:  50
第二次筛选:  45
第三次筛选:  23
20200423 【已筛选】优秀PPT模板收集（112页）/欧美黑白极简PPT模板动态-01（3页）/lo2.xml.json  text_type  0  tuwen_layout  1  rgraph_type  0
20200508 【已筛选】优秀PPT模板收集（135页）/欧美高端精选 (21)-（35页）/lo4.xml.json  text_type  0  tuwen_layout  1  rgraph_type  0
20200508 【已筛选】其他-4（94页）/逼格PPT-【NEW】black+for+free（20页）/lo3.xml.json  text_type  0  tuwen_layout  1  rgraph_type  0
20200429 【已筛选】优秀PPT模板收集（125页）/莫兰迪风尚-设计学长(4)-（6页）/lo3.xml.json  text_type  0  tuwen_layout  1  rgraph_type  0
20200426 【已筛选】优秀PPT模板收集（126页）/欧美黑白极简PPT模板静态-17（5页）/lo2.xml.json  text_type  0  tuwen_layout  1  rgraph_type  0
20200429 【已筛选】优秀PPT模板收集（125页）/莫兰迪风尚-设计学长(9)-（25页）/lo15.xml.json  text_type  0  tuwen_layout

{'text': {'input': [{'id': 0}], 'output': [{'id': 2, 'font-size': 20}]},
 'image': {'input': [{'id': 0}], 'output': [{'id': 1}]}}

In [ ]:
a

In [ ]:
b